### Import Required Libraries

In [2]:
from pymongo import MongoClient, ASCENDING, DESCENDING, TEXT
from datetime import datetime, timedelta
import pandas as pd
from bson import json_util
import json
import time
from pymongo.errors import OperationFailure, DuplicateKeyError
from pprint import pprint


## Part 1: Database Setup and Data Modeling (20 points)

#### Task 1.1: Create Database and Collections

In [13]:
# Establish connection
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

def create_collections_with_validation():
   
    # List of all collections with their validation schemas
    collections = {
        "users": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"],
                "properties": {
                    "userId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "email": {
                        "bsonType": "string",
                        "pattern": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$",
                        "description": "must be a valid email and is required"
                    },
                    "firstName": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "lastName": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "role": {
                        "enum": ["student", "instructor"],
                        "description": "must be either 'student' or 'instructor' and is required"
                    },
                    "dateJoined": {
                        "bsonType": "date",
                        "description": "must be a date and is required"
                    },
                    "profile": {
                        "bsonType": "object",
                        "properties": {
                            "bio": {"bsonType": "string"},
                            "avatar": {"bsonType": "string"},
                            "skills": {
                                "bsonType": "array",
                                "items": {"bsonType": "string"}
                            }
                        }
                    },
                    "isActive": {
                        "bsonType": "bool",
                        "description": "must be a boolean"
                    }
                }
            }
        },

        "courses": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ['courseId', 'title', 'description', 'instructorId', 'category', 
                     'level', 'duration', 'price', 'createdAt', 'isPublished'],
                "properties": {
                    "courseId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "title": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "description": {"bsonType": "string"},
                    "instructorId": {
                        "bsonType": "string",
                        "description": "must reference a user and is required"
                    },
                    "category": {"bsonType": "string"},
                    "level": {
                        "enum": ["beginner", "intermediate", "advanced"],
                        "description": "must be one of the defined levels"
                    },
                    "duration": {
                        "bsonType": "number",
                        "minimum": 0,
                        "description": "must be a positive number"
                    },
                    "price": {
                        "bsonType": "number",
                        "minimum": 0,
                        "description": "must be a positive number"
                    },
                    "tags": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    },
                    "createdAt": {
                        "bsonType": "date",
                        "description": "must be a date and is required"
                    },
                    "updatedAt": {"bsonType": "date"},
                    "isPublished": {"bsonType": "bool"}
                }
            }
        },

        "enrollments": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ['enrollmentId', 'studentId', 'courseId', 'enrollmentDate', 
                    'completionStatus', 'lastAccessed'],
                "properties": {
                    "enrollmentId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "studentId": {
                        "bsonType": "string",
                        "description": "must reference a user and is required"
                    },
                    "courseId": {
                        "bsonType": "string",
                        "description": "must reference a course and is required"
                    },
                    "enrollmentDate": {
                        "bsonType": "date",
                        "description": "must be a date and is required"
                    },
                    "completionStatus": {
                        "bsonType": "number",
                        "minimum": 0,
                        "maximum": 100,
                        "description": "must be a percentage between 0 and 100"
                    },
                    "lastAccessed": {"bsonType": "date"}
                }
            }
        },
        
        "lessons": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ['lessonId', 'courseId', 'title', 'content', 'sequence', 'duration'],
                "properties": {
                    "lessonId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "courseId": {
                        "bsonType": "string",
                        "description": "must reference a course and is required"
                    },
                    "title": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "content": {"bsonType": "string"},
                    "sequence": {
                        "bsonType": "number",
                        "minimum": 1,
                        "description": "must be a positive integer and is required"
                    },
                    "duration": {
                        "bsonType": "number",
                        "minimum": 0,
                        "description": "must be a positive number"
                    },
                    "resources": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    }
                }
            }
        },

        "assignments": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ['assignmentId', 'courseId', 'title', 'description', 'dueDate', 'maxPoints', 'instructions'],
                "properties": {
                    "assignmentId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "courseId": {
                        "bsonType": "string",
                        "description": "must reference a course and is required"
                    },
                    "title": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "description": {"bsonType": "string"},
                    "dueDate": {
                        "bsonType": "date",
                        "description": "must be a date and is required"
                    },
                    "maxPoints": {
                        "bsonType": "number",
                        "minimum": 0,
                        "description": "must be a positive number"
                    },
                    "instructions": {"bsonType": "string"}
                }
            }
        },

        "submissions": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ['submissionId', 'assignmentId', 'studentId', 'submittedDate', 'content', 'isGraded'],
                "properties": {
                    "submissionId": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "assignmentId": {
                        "bsonType": "string",
                        "description": "must reference an assignment and is required"
                    },
                    "studentId": {
                        "bsonType": "string",
                        "description": "must reference a user and is required"
                    },
                    "submittedDate": {
                        "bsonType": "date",
                        "description": "must be a date and is required"
                    },
                    "content": {"bsonType": "string"},
                    "grade": {
                        "bsonType": "number",
                        "minimum": 0,
                        "description": "must be a positive number"
                    },
                    "feedback": {"bsonType": "string"},
                    "isGraded": {"bsonType": "bool"}
                }
            }
        },
    }

    # Get list of existing collections
    existing_collections = db.list_collection_names()
    
    # Create each collection if it doesn't exist
    for collection_name, validator in collections.items():
        if collection_name not in existing_collections:
            db.create_collection(collection_name, validator=validator)
            print(f"Collection '{collection_name}' created with validation rules.")
        else:
            print(f"Collection '{collection_name}' already exists. Skipping creation.")

# Execute the function to create collections
create_collections_with_validation()

Collection 'users' created with validation rules.
Collection 'courses' created with validation rules.
Collection 'enrollments' created with validation rules.
Collection 'lessons' created with validation rules.
Collection 'assignments' created with validation rules.
Collection 'submissions' created with validation rules.


#### Verify that the collections exists

In [14]:
db.list_collection_names()

['submissions', 'courses', 'assignments', 'lessons', 'enrollments', 'users']

#### Task 1.2: Design Documet Schemas

In [15]:
# Create sample documents from the sample_data.json file
with open("C:/Users/USER/Desktop/mongodb-eduhub-project/data/sample_data.json") as f:
    sample_data = json.load(f)

users_sample_document = sample_data.get("users", [])
courses_sample_document = sample_data.get("courses", [])
enrollments_sample_document = sample_data.get("enrollments", [])
lessons_sample_document = sample_data.get("lessons", [])
assignments_sample_document = sample_data.get("assignments", [])
submissions_sample_document = sample_data.get("submissions", [])

# Print the first elements of each sample data
print(f"Users Sample Document: {json.dumps(users_sample_document[0], indent=4)}")
print("  ")
print(f"Courses Sample Document: {json.dumps(courses_sample_document[0], indent=4)}")
print("  ")
print(f"Enrollments Sample Document: {json.dumps(enrollments_sample_document[0], indent=4)}")
print("  ")
print(f"Lessons Sample Document: {json.dumps(lessons_sample_document[0], indent=4)}")
print("  ")
print(f"Assignments Sample Document: {json.dumps(assignments_sample_document[0], indent=4)}")
print("  ")
print(f"Submissions Sample Document: {json.dumps(submissions_sample_document[0], indent=4)}")

Users Sample Document: {
    "userId": "user001",
    "email": "adebola.adesanya@gmail.com",
    "firstName": "Adebola",
    "lastName": "Adesanya",
    "role": "student",
    "dateJoined": "2024-01-15T00:00:00Z",
    "profile": {
        "bio": "Computer science student at University of Lagos",
        "skills": [
            "Python",
            "Java"
        ]
    },
    "isActive": true
}
  
Courses Sample Document: {
    "courseId": "course001",
    "title": "Introduction to Python Programming",
    "description": "Learn Python programming from scratch with practical examples",
    "instructorId": "user002",
    "category": "Programming",
    "level": "beginner",
    "duration": 20,
    "price": 99.99,
    "tags": [
        "python",
        "programming",
        "beginner"
    ],
    "createdAt": "2024-01-01T00:00:00Z",
    "updatedAt": "2024-01-10T00:00:00Z",
    "isPublished": true
}
  
Enrollments Sample Document: {
    "enrollmentId": "enroll001",
    "studentId": "user001

### Part 2: Data Population (15 points)

#### Task 2.1: Insert Sample Data

In [16]:

def load_data_to_collections(json_file_path):
    # Connect to MongoDB
    # client = MongoClient('mongodb://localhost:27017/')
    # db = client['eduhub_db']
    
    # Load JSON data
    with open(json_file_path) as file:
        data = json.load(file)
    
    # Date fields for each collection
    date_fields = {
        'users': ['dateJoined'],
        'courses': ['createdAt', 'updatedAt'],
        'enrollments': ['enrollmentDate', 'lastAccessed'],
        'assignments': ['dueDate'],
        'submissions': ['submittedDate']
    }
    
    # Convert date strings to datetime objects
    def convert_dates(document, fields):
        for field in fields:
            if field in document and document[field]:
                document[field] = datetime.strptime(document[field], "%Y-%m-%dT%H:%M:%SZ")
        return document
    
    # Load data into each collection
    for collection_name in ['users', 'courses', 'enrollments', 'lessons', 'assignments', 'submissions']:
        if collection_name in data:
            collection = db[collection_name]
            
            # Convert dates for documents in this collection
            documents = data[collection_name]
            if collection_name in date_fields:
                documents = [convert_dates(doc, date_fields[collection_name]) for doc in documents]
            
            # Insert documents
            if documents:  # Only insert if there are documents
                result = collection.insert_many(documents)
                print(f"Inserted {len(result.inserted_ids)} documents into {collection_name} collection")
    
    print("Data loading completed!")

# Usage example:
load_data_to_collections('C:/Users/USER/Desktop/mongodb-eduhub-project/data/sample_data.json')

Inserted 20 documents into users collection
Inserted 8 documents into courses collection
Inserted 17 documents into enrollments collection
Inserted 25 documents into lessons collection
Inserted 10 documents into assignments collection
Inserted 15 documents into submissions collection
Data loading completed!


### Part 3: Basic CRUD Operations (25 points)

#### Task 3.1: Create Operations

Write Python code using PyMongo to perform the following create operations:

1. Add a new student user

2. Create a new course 

3. Enroll a student in a course

4. Add a new lesson to an existing course

In [17]:
# 1. Add a new student user
new_student = {
    "userId": "user021",
    "email": "new.student@example.com",
    "firstName": "New",
    "lastName": "Student",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "New computer science student",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}

student_result = db.users.insert_one(new_student)
print(f"1. Added new student (ID: {student_result.inserted_id}):")
print(f"   - Name: {new_student['firstName']} {new_student['lastName']}")
print(f"   - Email: {new_student['email']}\n")

# 2. Create a new course
new_course = {
    "courseId": "course009",
    "title": "Advanced Data Analysis",
    "description": "Master data analysis techniques with Python and Pandas",
    "instructorId": "user012",  # Nneka Onyemaobi (ML specialist)
    "category": "Data Science",
    "level": "advanced",
    "duration": 35,
    "price": 229.99,
    "tags": ["data analysis", "python", "pandas"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}

course_result = db.courses.insert_one(new_course)
print(f"2. Created new course (ID: {course_result.inserted_id}):")
print(f"   - Title: {new_course['title']}")
print(f"   - Instructor ID: {new_course['instructorId']}")
print(f"   - Price: ${new_course['price']}\n")

# 3. Enroll a student in a course
new_enrollment = {
    "enrollmentId": "enroll017",
    "studentId": "user021",  # Our new student
    "courseId": "course009",  # Our new course
    "enrollmentDate": datetime.now(),
    "completionStatus": 0,
    "lastAccessed": datetime.now()
}

enrollment_result = db.enrollments.insert_one(new_enrollment)
print(f"3. Created new enrollment (ID: {enrollment_result.inserted_id}):")
print(f"   - Student: {new_enrollment['studentId']}")
print(f"   - Course: {new_enrollment['courseId']}")
print(f"   - Status: {new_enrollment['completionStatus']}% complete\n")

# 4. Add a new lesson to an existing course
new_lesson = {
    "lessonId": "lesson026",
    "courseId": "course009",  # Our new course
    "title": "Pandas Advanced Features",
    "content": "Master multi-indexing, groupby operations, and performance optimization",
    "sequence": 1,
    "duration": 90,
    "resources": ["https://example.com/pandas-advanced"]
}

lesson_result = db.lessons.insert_one(new_lesson)
print(f"4. Added new lesson (ID: {lesson_result.inserted_id}):")
print(f"   - Course: {new_lesson['courseId']}")
print(f"   - Title: {new_lesson['title']}")
print(f"   - Duration: {new_lesson['duration']} minutes")

# Verification queries
print("\n=== Verification ===")
print(f"Total users: {db.users.count_documents({})}")
print(f"Total courses: {db.courses.count_documents({})}")
print(f"Total enrollments: {db.enrollments.count_documents({})}")
print(f"Total lessons: {db.lessons.count_documents({})}")

1. Added new student (ID: 684f21f98a7c67551054e541):
   - Name: New Student
   - Email: new.student@example.com

2. Created new course (ID: 684f21f98a7c67551054e542):
   - Title: Advanced Data Analysis
   - Instructor ID: user012
   - Price: $229.99

3. Created new enrollment (ID: 684f21f98a7c67551054e543):
   - Student: user021
   - Course: course009
   - Status: 0% complete

4. Added new lesson (ID: 684f21f98a7c67551054e544):
   - Course: course009
   - Title: Pandas Advanced Features
   - Duration: 90 minutes

=== Verification ===
Total users: 21
Total courses: 9
Total enrollments: 18
Total lessons: 26


#### Task 3.2: Read Operations

Write Python queries to:

1. Find all active students

2. Retrieve course details with instructor information

3. Get all courses in a specific category

4. Find students enrolled in a particular course

5. Search courses by title (case-insensitive, partial match).

In [18]:

# 1. Find all active students
active_students = list(db.users.find({
    "role": "student",
    "isActive": True
}, {"userId": 1, "firstName": 1, "lastName": 1, "email": 1}))

print("1. Active Students (Total:", len(active_students), "):")
for student in active_students[:3]:  # Display first 3 for brevity
    print(f"   - {student['firstName']} {student['lastName']} ({student['email']})")
print("   ...\n")

# 2. Retrieve course details with instructor information
course_with_instructor = list(db.courses.aggregate([
    {
        "$match": {"courseId": "course001"}  # Python course
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {
        "$unwind": "$instructor"
    },
    {
        "$project": {
            "title": 1,
            "description": 1,
            "level": 1,
            "instructorName": {"$concat": ["$instructor.firstName", " ", "$instructor.lastName"]},
            "instructorBio": "$instructor.profile.bio"
        }
    }
]))

print("2. Course with Instructor Details:")
pprint(course_with_instructor[0])
print("\n")

# 3. Get all courses in a specific category
data_science_courses = list(db.courses.find(
    {"category": "Data Science"},
    {"title": 1, "level": 1, "price": 1}
))

print("3. Data Science Courses (Total:", len(data_science_courses), "):")
for course in data_science_courses:
    print(f"   - {course['title']} ({course['level']}, ${course['price']})")
print("\n")

# 4. Find students enrolled in a particular course
python_students = list(db.enrollments.aggregate([
    {
        "$match": {"courseId": "course001"}  # Python course
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "studentId",
            "foreignField": "userId",
            "as": "student"
        }
    },
    {
        "$unwind": "$student"
    },
    {
        "$project": {
            "studentName": {"$concat": ["$student.firstName", " ", "$student.lastName"]},
            "email": "$student.email",
            "completionStatus": 1
        }
    }
]))

print("4. Students Enrolled in Python Course (Total:", len(python_students), "):")
for student in python_students[:3]:  # Display first 3
    print(f"   - {student['studentName']} ({student['email']}), Progress: {student['completionStatus']}%")
print("   ...\n")

# 5. Search courses by title (case-insensitive partial match)
search_term = "data"
matched_courses = list(db.courses.find(
    {"title": {"$regex": search_term, "$options": "i"}},
    {"title": 1, "category": 1}
))

print(f"5. Courses matching '{search_term}' (Total:", len(matched_courses), "):")
for course in matched_courses:
    print(f"   - {course['title']} ({course['category']})")

# Verification counts
print("\n=== Verification Counts ===")
print("Total active students:", db.users.count_documents({"role": "student", "isActive": True}))
print("Total Data Science courses:", db.courses.count_documents({"category": "Data Science"}))
print("Total enrollments in Python course:", db.enrollments.count_documents({"courseId": "course001"}))

1. Active Students (Total: 14 ):
   - Adebola Adesanya (adebola.adesanya@gmail.com)
   - Emeka Eze (emeka.eze@gmail.com)
   - Gbenga Oladele (gbenga.oladele@gmail.com)
   ...

2. Course with Instructor Details:
{'_id': ObjectId('684f21da8a7c67551054e4f6'),
 'description': 'Learn Python programming from scratch with practical examples',
 'instructorBio': 'Senior software engineer with 10 years experience',
 'instructorName': 'Chinwe Okonkwo',
 'level': 'beginner',
 'title': 'Introduction to Python Programming'}


3. Data Science Courses (Total: 2 ):
   - Machine Learning with Python (intermediate, $249.99)
   - Advanced Data Analysis (advanced, $229.99)


4. Students Enrolled in Python Course (Total: 4 ):
   - Adebola Adesanya (adebola.adesanya@gmail.com), Progress: 25%
   - Emeka Eze (emeka.eze@gmail.com), Progress: 40%
   - Tunde Adeleke (tunde.adeleke@yahoo.com), Progress: 20%
   ...

5. Courses matching 'data' (Total: 2 ):
   - Database Design with MongoDB (Database)
   - Advanced D

### Task 3.3 Update Operations

**Write Python code to:**

1. Update a user’s profile information

2. Mark a course as published

3. Update assignment grades

4. Add tags to an existing course.

In [19]:


# 1. Update a user's profile information
user_update_result = db.users.update_one(
    {"userId": "user001"},  # Adebola Adesanya
    {
        "$set": {
            "profile.bio": "Computer science graduate specializing in AI",
            "profile.skills": ["Python", "Java", "Machine Learning"],
            "lastName": "Adesanya-Jones"  # Married name change
        }
    }
)

# Verification
updated_user = db.users.find_one({"userId": "user001"})
print("1. Updated User Profile:")
print(f"   - Name: {updated_user['firstName']} {updated_user['lastName']}")
print(f"   - New Bio: {updated_user['profile']['bio']}")
print(f"   - Skills: {', '.join(updated_user['profile']['skills'])}")
print(f"   - Documents modified: {user_update_result.modified_count}\n")

# 2. Mark a course as published
course_publish_result = db.courses.update_one(
    {"courseId": "course008"},  # Database Design course
    {
        "$set": {
            "isPublished": True,
            "updatedAt": datetime.utcnow()
        }
    }
)

# Verification
published_course = db.courses.find_one({"courseId": "course008"})
print("2. Course Publishing Status:")
print(f"   - Course: {published_course['title']}")
print(f"   - Published: {published_course['isPublished']}")
print(f"   - Last Updated: {published_course['updatedAt']}")
print(f"   - Documents modified: {course_publish_result.modified_count}\n")

# 3. Update assignment grades
grade_update_result = db.submissions.update_one(
    {
        "submissionId": "sub002",  # Control flow assignment
        "studentId": "user001"     # Adebola's submission
    },
    {
        "$set": {
            "grade": 95,  # Original was 90
            "feedback": "Excellent work! Fixed all edge cases.",
            "isGraded": True
        }
    }
)

# Verification
updated_submission = db.submissions.find_one({"submissionId": "sub002"})
print("3. Updated Assignment Grade:")
print(f"   - Student: {updated_submission['studentId']}")
print(f"   - New Grade: {updated_submission['grade']}/100")
print(f"   - Feedback: {updated_submission['feedback']}")
print(f"   - Documents modified: {grade_update_result.modified_count}\n")

# 4. Add tags to an existing course
tags_update_result = db.courses.update_one(
    {"courseId": "course005"},  # Machine Learning course
    {
        "$addToSet": {  # Prevents duplicate tags
            "tags": {"$each": ["deep learning", "neural networks"]}
        },
        "$set": {
            "updatedAt": datetime.utcnow()
        }
    }
)

# Verification
updated_course = db.courses.find_one({"courseId": "course005"})
print("4. Added Course Tags:")
print(f"   - Course: {updated_course['title']}")
print(f"   - Updated Tags: {', '.join(updated_course['tags'])}")
print(f"   - Last Updated: {updated_course['updatedAt']}")
print(f"   - Documents modified: {tags_update_result.modified_count}")

# Final verification counts
print("\n=== Final Verification ===")
print("User profile changes:", user_update_result.modified_count)
print("Courses published:", course_publish_result.modified_count)
print("Grades updated:", grade_update_result.modified_count)
print("Courses with new tags:", tags_update_result.modified_count)

1. Updated User Profile:
   - Name: Adebola Adesanya-Jones
   - New Bio: Computer science graduate specializing in AI
   - Skills: Python, Java, Machine Learning
   - Documents modified: 1

2. Course Publishing Status:
   - Course: Database Design with MongoDB
   - Published: True
   - Last Updated: 2025-06-15 19:45:00.939000
   - Documents modified: 1

3. Updated Assignment Grade:
   - Student: user001
   - New Grade: 95/100
   - Feedback: Excellent work! Fixed all edge cases.
   - Documents modified: 1

4. Added Course Tags:
   - Course: Machine Learning with Python
   - Updated Tags: machine learning, python, ai, deep learning, neural networks
   - Last Updated: 2025-06-15 19:45:00.944000
   - Documents modified: 1

=== Final Verification ===
User profile changes: 1
Courses published: 1
Grades updated: 1
Courses with new tags: 1


#### Task 3.4 Delete Operations

Write Python code to:

1. Remove a user (soft delete by setting isActive to false)

2. Delete an enrollment

3. Remove a lesson from a course.

In [20]:
# 1. Soft delete a user (set isActive to false)
soft_delete_result = db.users.update_one(
    {"userId": "user020"},  # Victoria Olumide
    {"$set": {"isActive": False}}
)

# Verification
deleted_user = db.users.find_one({"userId": "user020"})
print("1. Soft Deleted User:")
print(f"   - Name: {deleted_user['firstName']} {deleted_user['lastName']}")
print(f"   - isActive Status: {deleted_user['isActive']}")
print(f"   - Documents modified: {soft_delete_result.modified_count}")
print(f"   - Active users count: {db.users.count_documents({'isActive': True, 'role': 'student'})}\n")

# 2. Delete an enrollment
enrollment_delete_result = db.enrollments.delete_one(
    {"enrollmentId": "enroll016"}  # Course002 enrollment
)

# Verification
print("2. Deleted Enrollment:")
print(f"   - Enrollment ID: enroll016")
print(f"   - Documents deleted: {enrollment_delete_result.deleted_count}")
print(f"   - Remaining enrollments: {db.enrollments.count_documents({})}")
print(f"   - Course002 enrollments: {db.enrollments.count_documents({'courseId': 'course002'})}\n")

# 3. Remove a lesson from a course
lesson_remove_result = db.lessons.delete_one(
    {
        "lessonId": "lesson025",  # Python Data Structures
        "courseId": "course001"   # From Python course
    }
)

# Verification
print("3. Removed Lesson:")
print(f"   - Lesson ID: lesson025")
print(f"   - Documents deleted: {lesson_remove_result.deleted_count}")
print(f"   - Remaining lessons: {db.lessons.count_documents({})}")
print(f"   - Lessons in Python course: {db.lessons.count_documents({'courseId': 'course001'})}")

# Final verification
print("\n=== Final Verification ===")
print(f"Active students count: {db.users.count_documents({'role': 'student', 'isActive': True})}")
print(f"Total enrollments: {db.enrollments.count_documents({})}")
print(f"Total lessons: {db.lessons.count_documents({})}")

# Additional verification queries
print("\n=== Detailed Verification ===")
print("User020 status:", db.users.find_one({"userId": "user020"}, {"isActive": 1}))
print("Enrollment016 exists:", bool(db.enrollments.find_one({"enrollmentId": "enroll016"})))
print("Lesson025 exists:", bool(db.lessons.find_one({"lessonId": "lesson025"})))

1. Soft Deleted User:
   - Name: Victoria Olumide
   - isActive Status: False
   - Documents modified: 1
   - Active users count: 13

2. Deleted Enrollment:
   - Enrollment ID: enroll016
   - Documents deleted: 1
   - Remaining enrollments: 17
   - Course002 enrollments: 2

3. Removed Lesson:
   - Lesson ID: lesson025
   - Documents deleted: 1
   - Remaining lessons: 25
   - Lessons in Python course: 3

=== Final Verification ===
Active students count: 13
Total enrollments: 17
Total lessons: 25

=== Detailed Verification ===
User020 status: {'_id': ObjectId('684f21da8a7c67551054e4f5'), 'isActive': False}
Enrollment016 exists: False
Lesson025 exists: False


### Part 4: Advanced Queries and Aggregation (25 points)

#### Task 4.1 Complex Queries

Write Python code using various PyMongo operators:

1. Find courses with price between $50 and $200

2. Get users who joined in the last 6 months

3. Find courses that have specific tags using $in operator

4. Retrieve assignments with due dates in the next week.

In [21]:
# 1. Find courses with price between $50 and $200
price_range_courses = list(db.courses.find(
    {
        "price": {
            "$gte": 50,
            "$lte": 200
        }
    },
    {
        "title": 1,
        "price": 1,
        "category": 1,
        "_id": 0
    }
).sort("price", 1))  # Sort by price ascending

print("1. Courses between $50-$200 (Count:", len(price_range_courses), "):")
for course in price_range_courses:
    print(f"   - {course['title']}: ${course['price']} ({course['category']})")
print()

# 2. Get users who joined in the last 6 months
six_months_ago = datetime.now() - timedelta(days=180)
recent_users = list(db.users.find(
    {
        "dateJoined": {"$gte": six_months_ago},
        "role": "student"
    },
    {
        "firstName": 1,
        "lastName": 1,
        "dateJoined": 1,
        "_id": 0
    }
).sort("dateJoined", -1))  # Newest first

print("2. Students joined in last 6 months (Count:", len(recent_users), "):")
for user in recent_users[:3]:  # Show first 3 for brevity
    join_date = user['dateJoined'].strftime("%Y-%m-%d")
    print(f"   - {user['firstName']} {user['lastName']} (Joined: {join_date})")
print("   ...\n")

# 3. Find courses that have specific tags using $in operator
target_tags = ["python", "machine learning"]
tagged_courses = list(db.courses.find(
    {
        "tags": {"$in": target_tags}
    },
    {
        "title": 1,
        "tags": 1,
        "_id": 0
    }
))

print(f"3. Courses with tags {target_tags} (Count:", len(tagged_courses), "):")
for course in tagged_courses:
    matched_tags = [tag for tag in course['tags'] if tag in target_tags]
    print(f"   - {course['title']} (Tags: {', '.join(matched_tags)})")
print()

# 4. Retrieve assignments with due dates in the next week
today = datetime.now()
next_week = today + timedelta(days=7)
upcoming_assignments = list(db.assignments.find(
    {
        "dueDate": {
            "$gte": today,
            "$lte": next_week
        }
    },
    {
        "title": 1,
        "courseId": 1,
        "dueDate": 1,
        "_id": 0
    }
).sort("dueDate", 1))  # Earliest first

print("4. Assignments due in next week (Count:", len(upcoming_assignments), "):")
for assignment in upcoming_assignments:
    due_date = assignment['dueDate'].strftime("%Y-%m-%d")
    course = db.courses.find_one(
        {"courseId": assignment['courseId']},
        {"title": 1}
    )
    print(f"   - {assignment['title']} (Due: {due_date})")
    print(f"     Course: {course['title']}")
print()

# Verification counts
print("=== Verification Counts ===")
print("Courses $50-$200:", len(price_range_courses))
print("Recent students:", len(recent_users))
print(f"Courses with tags {target_tags}:", len(tagged_courses))
print("Upcoming assignments:", len(upcoming_assignments))

# Sample document verification
print("\n=== Sample Document Verification ===")
print("Sample course in price range:", price_range_courses[0] if price_range_courses else "None")
print("Most recent student:", recent_users[0] if recent_users else "None")
print("Sample tagged course:", tagged_courses[0] if tagged_courses else "None")
print("Next due assignment:", upcoming_assignments[0] if upcoming_assignments else "None")

1. Courses between $50-$200 (Count: 6 ):
   - Introduction to Python Programming: $99.99 (Programming)
   - Cybersecurity Fundamentals: $129.99 (Security)
   - Web Development with Django: $149.99 (Web Development)
   - Database Design with MongoDB: $159.99 (Database)
   - Mobile App Development with Flutter: $179.99 (Mobile Development)
   - AWS Cloud Practitioner: $199.99 (Cloud Computing)

2. Students joined in last 6 months (Count: 1 ):
   - New Student (Joined: 2025-06-15)
   ...

3. Courses with tags ['python', 'machine learning'] (Count: 4 ):
   - Introduction to Python Programming (Tags: python)
   - Web Development with Django (Tags: python)
   - Machine Learning with Python (Tags: machine learning, python)
   - Advanced Data Analysis (Tags: python)

4. Assignments due in next week (Count: 0 ):

=== Verification Counts ===
Courses $50-$200: 6
Recent students: 1
Courses with tags ['python', 'machine learning']: 4
Upcoming assignments: 0

=== Sample Document Verification ===
Sam

#### Task 4.1 Aggregation Pipeline

1. **Course Enrollment Statistics**

+ Count the total enrollment per course

+ Calculate average course rating

+ Group by course category

In [23]:

# 1. Course Enrollment Statistics Pipeline
enrollment_stats = db.enrollments.aggregate([
    # Join with courses collection
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    
    # Join with submissions for ratings (assuming rating is in submissions)
    {
        "$lookup": {
            "from": "submissions",
            "localField": "studentId",
            "foreignField": "studentId",
            "as": "submissions"
        }
    },
    
    # Group by course and calculate metrics
    {
        "$group": {
            "_id": {
                "courseId": "$courseId",
                "title": "$course.title",
                "category": "$course.category"
            },
            "totalEnrollments": {"$sum": 1},
            "averageGrade": {"$avg": "$submissions.grade"},
            "completionRate": {"$avg": "$completionStatus"}
        }
    },
    
    # Project for cleaner output
    {
        "$project": {
            "courseTitle": "$_id.title",
            "category": "$_id.category",
            "totalEnrollments": 1,
            "averageGrade": {"$round": ["$averageGrade", 2]},
            "averageCompletion": {"$round": ["$completionRate", 2]},
            "_id": 0
        }
    },
    
    # Sort by enrollments (descending)
    {"$sort": {"totalEnrollments": -1}}
])

# Convert to list for display
stats_list = list(enrollment_stats)

print("1. Course Enrollment Statistics:")
print("=" * 50)
pprint(stats_list)

# Display as a table using pandas
print("\nFormatted Results:")
print("=" * 100)
df = pd.DataFrame(stats_list)
print(df.to_string(index=False))

# Verification metrics
total_courses = db.courses.count_documents({})
print("\n=== Verification ===")
print(f"Total courses in system: {total_courses}")
print(f"Courses with enrollment data: {len(stats_list)}")
print(f"Sample course stats:")
pprint(stats_list[0] if stats_list else "No results")

# Additional verification queries
most_popular = max(stats_list, key=lambda x: x['totalEnrollments']) if stats_list else None
print("\nMost Popular Course:")
print(f" - Title: {most_popular['courseTitle'] if most_popular else 'N/A'}")
print(f" - Enrollments: {most_popular['totalEnrollments'] if most_popular else 'N/A'}")
print(f" - Avg Grade: {most_popular['averageGrade'] if most_popular else 'N/A'}")

# Compare with raw counts
print("\nRaw Enrollment Counts per Course:")
raw_counts = db.enrollments.aggregate([
    {"$group": {"_id": "$courseId", "count": {"$sum": 1}}}
])
for course in raw_counts:
    c = db.courses.find_one({"courseId": course["_id"]}, {"title": 1})
    print(f" - {c['title'] if c else 'Unknown'}: {course['count']}")

1. Course Enrollment Statistics:
[{'averageCompletion': 28.75,
  'averageGrade': None,
  'category': 'Programming',
  'courseTitle': 'Introduction to Python Programming',
  'totalEnrollments': 4},
 {'averageCompletion': 38.33,
  'averageGrade': None,
  'category': 'Security',
  'courseTitle': 'Cybersecurity Fundamentals',
  'totalEnrollments': 3},
 {'averageCompletion': 80.0,
  'averageGrade': None,
  'category': 'Cloud Computing',
  'courseTitle': 'AWS Cloud Practitioner',
  'totalEnrollments': 3},
 {'averageCompletion': 40.0,
  'averageGrade': None,
  'category': 'Web Development',
  'courseTitle': 'Web Development with Django',
  'totalEnrollments': 2},
 {'averageCompletion': 20.0,
  'averageGrade': None,
  'category': 'Blockchain',
  'courseTitle': 'Blockchain Development Basics',
  'totalEnrollments': 1},
 {'averageCompletion': 5.0,
  'averageGrade': None,
  'category': 'Database',
  'courseTitle': 'Database Design with MongoDB',
  'totalEnrollments': 1},
 {'averageCompletion': 0.

2. **Students Performance Analysis:**

+ Average grade per student

+ Completion rate per student

+ Top-performing Students

In [24]:

student_performance = db.enrollments.aggregate([
    # Join with users collection
    {
        "$lookup": {
            "from": "users",
            "localField": "studentId",
            "foreignField": "userId",
            "as": "student"
        }
    },
    {"$unwind": "$student"},
    
    # Join with submissions (preserving enrollments without submissions)
    {
        "$lookup": {
            "from": "submissions",
            "localField": "studentId",
            "foreignField": "studentId",
            "as": "submissions"
        }
    },
    
    # Add field to check if student has submissions
    {
        "$addFields": {
            "hasSubmissions": {"$gt": [{"$size": "$submissions"}, 0]}
        }
    },
    
    # Calculate average grade per enrollment (handling empty arrays)
    {
        "$addFields": {
            "enrollmentAvgGrade": {
                "$cond": [
                    {"$eq": [{"$size": "$submissions"}, 0]},
                    None,
                    {"$avg": "$submissions.grade"}
                ]
            }
        }
    },
    
    # Group by student
    {
        "$group": {
            "_id": {
                "studentId": "$studentId",
                "name": {"$concat": ["$student.firstName", " ", "$student.lastName"]}
            },
            "coursesEnrolled": {"$sum": 1},
            "coursesWithSubmissions": {"$sum": {"$cond": ["$hasSubmissions", 1, 0]}},
            "averageGrade": {"$avg": "$enrollmentAvgGrade"},
            "averageCompletion": {"$avg": "$completionStatus"},
            "submissionCount": {"$sum": {"$size": "$submissions"}}
        }
    },
    
    # Format output
    {
        "$project": {
            "studentName": "$_id.name",
            "coursesEnrolled": 1,
            "coursesWithSubmissions": 1,
            "averageGrade": {
                "$ifNull": [
                    {"$round": ["$averageGrade", 2]},
                    None
                ]
            },
            "averageCompletion": {"$round": ["$averageCompletion", 2]},
            "submissionCount": 1,
            "_id": 0
        }
    },
    
    # Sort by average grade (descending), putting nulls last
    {
        "$sort": {
            "averageGrade": -1,
            "submissionCount": -1
        }
    }
])

# Convert to list and display
performance_data = list(student_performance)

print("Student Performance Analysis:")
print("=" * 60)
pprint(performance_data)

# Display as table
print("\nTop Performing Students:")
print("=" * 120)
df = pd.DataFrame(performance_data)
print(df.to_string(index=False))

# 2. Completion Rate by Course
completion_by_course = db.enrollments.aggregate([
    # Join with courses
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    
    # Group by course
    {
        "$group": {
            "_id": {
                "courseId": "$courseId",
                "title": "$course.title"
            },
            "averageCompletion": {"$avg": "$completionStatus"},
            "totalStudents": {"$sum": 1}
        }
    },
    
    # Format output
    {
        "$project": {
            "courseTitle": "$_id.title",
            "averageCompletion": {"$round": ["$averageCompletion", 2]},
            "totalStudents": 1,
            "_id": 0
        }
    },
    
    # Sort by completion rate
    {"$sort": {"averageCompletion": -1}}
])

print("\nCourse Completion Rates:")
print("=" * 60)
for course in completion_by_course:
    print(f"{course['courseTitle']}: {course['averageCompletion']}% ({course['totalStudents']} students)")

# Verification
print("\n=== Verification ===")
top_student = performance_data[0] if performance_data else None
print(f"Top student: {top_student['studentName'] if top_student else 'N/A'}")
print(f"Avg grade: {top_student['averageGrade'] if top_student else 'N/A'}")
print(f"Enrollments analyzed: {len(performance_data)}")

# Verify with raw data
sample_student = db.users.find_one({"userId": "user001"})
student_grades = list(db.submissions.find({"studentId": "user001"}, {"grade": 1}))
avg_grade = sum(g['grade'] for g in student_grades) / len(student_grades) if student_grades else 0

print("\nSample Student Verification:")
print(f"Name: {sample_student['firstName']} {sample_student['lastName']}")
print(f"Calculated avg grade: {round(avg_grade, 2)}")
print(f"Submissions: {len(student_grades)}")

Student Performance Analysis:
[{'averageCompletion': 15.0,
  'averageGrade': 95.0,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': 'Gbenga Oladele',
  'submissionCount': 1},
 {'averageCompletion': 25.0,
  'averageGrade': 94.5,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': 'Adebola Adesanya-Jones',
  'submissionCount': 2},
 {'averageCompletion': 20.0,
  'averageGrade': 93.0,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': 'Obinna Okafor',
  'submissionCount': 1},
 {'averageCompletion': 10.0,
  'averageGrade': 92.0,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': 'Jumoke Adebayo',
  'submissionCount': 1},
 {'averageCompletion': 50.0,
  'averageGrade': 89.0,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': 'Musa Ibrahim',
  'submissionCount': 1},
 {'averageCompletion': 5.0,
  'averageGrade': 88.0,
  'coursesEnrolled': 1,
  'coursesWithSubmissions': 1,
  'studentName': '

3. **Instructor Analytics:**

+ Total Student taught by each instructor

+ Average course rating per instructor

+ Revenue generated per instructor

In [25]:

# Instructor Analytics Pipeline
instructor_analytics = db.courses.aggregate([
    # Join with instructors
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {"$unwind": "$instructor"},
    
    # Join with enrollments
    {
        "$lookup": {
            "from": "enrollments",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "enrollments"
        }
    },
    
    # Join with submissions for ratings
    {
        "$lookup": {
            "from": "submissions",
            "localField": "enrollments.studentId",
            "foreignField": "studentId",
            "as": "submissions"
        }
    },
    
    # Calculate metrics per instructor-course combination
    {
        "$project": {
            "instructorId": 1,
            "instructorName": {"$concat": ["$instructor.firstName", " ", "$instructor.lastName"]},
            "courseTitle": "$title",
            "studentCount": {"$size": "$enrollments"},
            "courseRevenue": {"$multiply": ["$price", {"$size": "$enrollments"}]},
            "avgGrade": {"$avg": "$submissions.grade"}
        }
    },
    
    # Group by instructor
    {
        "$group": {
            "_id": "$instructorId",
            "instructorName": {"$first": "$instructorName"},
            "totalStudents": {"$sum": "$studentCount"},
            "totalRevenue": {"$sum": "$courseRevenue"},
            "coursesTaught": {"$sum": 1},
            "avgCourseRating": {"$avg": "$avgGrade"}
        }
    },
    
    # Format output
    {
        "$project": {
            "instructorName": 1,
            "totalStudents": 1,
            "totalRevenue": {"$round": ["$totalRevenue", 2]},
            "coursesTaught": 1,
            "avgCourseRating": {
                "$ifNull": [
                    {"$round": ["$avgCourseRating", 2]},
                    "No ratings yet"
                ]
            },
            "_id": 0
        }
    },
    
    # Sort by total students (descending)
    {"$sort": {"totalStudents": -1}}
])


# Convert to list and display
analytics_data = list(instructor_analytics)

print("Instructor Analytics:")
print("=" * 60)
pprint(analytics_data)

# Display as table
print("\nFormatted Results:")
print("=" * 60)
df = pd.DataFrame(analytics_data)
print(df.to_string(index=False))

# Verification
print("\n=== Verification ===")

# Verify Chinwe Okonkwo (user002) - teaches Python and Django courses
chinwe_courses = list(db.courses.find({"instructorId": "user002"}, {"courseId": 1, "title": 1, "price": 1}))
chinwe_enrollments = db.enrollments.count_documents({"courseId": {"$in": [c["courseId"] for c in chinwe_courses]}})
chinwe_revenue = sum(c["price"] * db.enrollments.count_documents({"courseId": c["courseId"]}) for c in chinwe_courses)

print(f"\nManual calculation for Chinwe Okonkwo:")
print(f"Courses taught: {[c['title'] for c in chinwe_courses]}")
print(f"Total students: {chinwe_enrollments}")
print(f"Calculated revenue: ${chinwe_revenue:.2f}")

# Check against aggregation results
chinwe_agg = next((i for i in analytics_data if i["instructorName"] == "Chinwe Okonkwo"), None)
print(f"\nAggregation results:")
print(f"Total students: {chinwe_agg['totalStudents'] if chinwe_agg else 'N/A'}")
print(f"Total revenue: ${chinwe_agg['totalRevenue'] if chinwe_agg else 'N/A'}")

Instructor Analytics:
[{'avgCourseRating': 90.62,
  'coursesTaught': 2,
  'instructorName': 'Chinwe Okonkwo',
  'totalRevenue': 699.94,
  'totalStudents': 6},
 {'avgCourseRating': 88.0,
  'coursesTaught': 1,
  'instructorName': 'Folake Adeleke',
  'totalRevenue': 599.97,
  'totalStudents': 3},
 {'avgCourseRating': 92.0,
  'coursesTaught': 1,
  'instructorName': 'Halima Yusuf',
  'totalRevenue': 389.97,
  'totalStudents': 3},
 {'avgCourseRating': 88.5,
  'coursesTaught': 2,
  'instructorName': 'Kabiru Mohammed',
  'totalRevenue': 339.98,
  'totalStudents': 2},
 {'avgCourseRating': 84.0,
  'coursesTaught': 2,
  'instructorName': 'Nneka Onyemaobi',
  'totalRevenue': 479.98,
  'totalStudents': 2},
 {'avgCourseRating': 93.0,
  'coursesTaught': 1,
  'instructorName': 'Quadri Bello',
  'totalRevenue': 299.99,
  'totalStudents': 1}]

Formatted Results:
 instructorName  totalStudents  coursesTaught  totalRevenue  avgCourseRating
 Chinwe Okonkwo              6              2        699.94       

4. **Advanced Analytics:**

+ Monthly enrollment trends

+ Most popular course categories

+ Student engagement metrics

In [26]:


# 1. Monthly Enrollment Trends
print("\n1. Monthly Enrollment Trends:")
monthly_enrollments = list(db.enrollments.aggregate([
    {
        "$group": {
            "_id": {
                "year": {"$year": "$enrollmentDate"},
                "month": {"$month": "$enrollmentDate"}
            },
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"_id.year": 1, "_id.month": 1}}
]))

for month in monthly_enrollments:
    print(f"{month['_id']['month']}/{month['_id']['year']}: {month['count']} enrollments")

# 2. Most Popular Course Categories
print("\n2. Most Popular Course Categories:")
popular_categories = list(db.enrollments.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.category",
            "enrollmentCount": {"$sum": 1}
        }
    },
    {"$sort": {"enrollmentCount": -1}}
]))

for category in popular_categories:
    print(f"{category['_id']}: {category['enrollmentCount']} enrollments")

# 3. Student Engagement Metrics
print("\n3. Student Engagement Metrics:")
total_enrollments = db.enrollments.count_documents({})
active_enrollments = db.enrollments.count_documents({"completionStatus": {"$gt": 0}})
submission_count = db.submissions.count_documents({})

print(f"Total enrollments: {total_enrollments}")
print(f"Active enrollments: {active_enrollments} ({round(active_enrollments/total_enrollments*100, 2)}%)")
print(f"Submissions made: {submission_count}")
print(f"Avg submissions per enrollment: {round(submission_count/total_enrollments, 2)}")

# Verification
print("\n=== Verification ===")
sample_course = db.courses.find_one({"category": "Programming"})
course_enrollments = db.enrollments.count_documents({"courseId": sample_course["courseId"]})
print(f"Sample course '{sample_course['title']}' has {course_enrollments} enrollments")


1. Monthly Enrollment Trends:
1/2024: 3 enrollments
2/2024: 5 enrollments
3/2024: 5 enrollments
4/2024: 3 enrollments
6/2025: 1 enrollments

2. Most Popular Course Categories:
Programming: 4 enrollments
Cloud Computing: 3 enrollments
Security: 3 enrollments
Web Development: 2 enrollments
Data Science: 2 enrollments
Database: 1 enrollments
Blockchain: 1 enrollments
Mobile Development: 1 enrollments

3. Student Engagement Metrics:
Total enrollments: 17
Active enrollments: 16 (94.12%)
Submissions made: 15
Avg submissions per enrollment: 0.88

=== Verification ===
Sample course 'Introduction to Python Programming' has 4 enrollments


### Part 5 Indexing and Performance (10 Points)

#### Task 5.1: Index Creation

Create appropriate indexes for:

1. User email lookup

2. Course search by title and category

3. Assignment queries by due date

4. Enrollment queries by student and course

In [27]:

# 1. User email lookup index
db.users.create_index([("email", ASCENDING)], unique=True, name="email_lookup_idx")
print("1. Created unique index on 'email' field in users collection")

# 2. Course search by title and category index
db.courses.create_index([
    ("title", TEXT),
    ("category", ASCENDING)
], name="course_search_idx")
print("2. Created compound index on 'title' (text) and 'category' in courses collection")

# 3. Assignment queries by due date index
db.assignments.create_index([("dueDate", ASCENDING)], name="due_date_idx")
print("3. Created index on 'dueDate' field in assignments collection")

# 4. Enrollment queries by student and course index
db.enrollments.create_index([
    ("studentId", ASCENDING),
    ("courseId", ASCENDING)
], name="student_course_idx")
print("4. Created compound index on 'studentId' and 'courseId' in enrollments collection")

# Verification function
def verify_index(collection, index_name):
    indexes = collection.index_information()
    return index_name in indexes

# Performance test function
# def test_query_performance(collection, query, index_name=None):
#     start = time.time()
#     list(collection.find(query).limit(100))
#     end = time.time()
#     return end - start

# Verify all indexes were created
print("\n=== Index Verification ===")
print(f"Email index exists: {verify_index(db.users, 'email_lookup_idx')}")
print(f"Course search index exists: {verify_index(db.courses, 'course_search_idx')}")
print(f"Due date index exists: {verify_index(db.assignments, 'due_date_idx')}")
print(f"Enrollment index exists: {verify_index(db.enrollments, 'student_course_idx')}")


1. Created unique index on 'email' field in users collection
2. Created compound index on 'title' (text) and 'category' in courses collection
3. Created index on 'dueDate' field in assignments collection
4. Created compound index on 'studentId' and 'courseId' in enrollments collection

=== Index Verification ===
Email index exists: True
Course search index exists: True
Due date index exists: True
Enrollment index exists: True


Task 5.2: Query Optimization

+ Analyze query performance using explain() method in PyMongo

+ Optimize at least 3 slow queries

+ Document the performance improvements using Python timing functions

In [28]:


def create_index_safely(collection, index_spec, index_name):
    """Create index if it doesn't already exist, or return existing matching index"""
    existing_indexes = collection.index_information()
    
    # Check if equivalent index exists
    for name, spec in existing_indexes.items():
        if spec['key'] == index_spec:
            print(f"Using existing index {name} with same specification")
            return name
    
    # If not, create new index
    try:
        collection.create_index(index_spec, name=index_name)
        print(f"Created new index: {index_name}")
        return index_name
    except OperationFailure as e:
        if e.code == 85:  # IndexOptionsConflict
            existing_name = next(name for name, spec in existing_indexes.items() 
                            if spec['key'] == index_spec)
            print(f"Using existing index {existing_name} (conflict with {index_name})")
            return existing_name
        raise

def test_query(query_func, query_name):
    """Test and analyze a query's performance"""
    # Run query and time it
    start_time = time.time()
    results = list(query_func())
    end_time = time.time()
    
    # Get execution stats
    explain = query_func().explain()
    execution_stats = explain.get('executionStats', explain)
    
    # Determine if index was used
    index_used = 'None'
    if 'indexName' in execution_stats:
        index_used = execution_stats['indexName']
    elif execution_stats.get('executionStages', {}).get('stage') == 'IXSCAN':
        index_used = execution_stats['executionStages'].get('indexName', 'Unknown')
    
    # Print results
    print(f"\n{query_name.upper()} RESULTS")
    print("-" * 40)
    print(f"Execution time: {(end_time - start_time)*1000:.2f} ms")
    print(f"Documents examined: {execution_stats.get('totalDocsExamined', 'N/A')}")
    print(f"Results returned: {len(results)}")
    print(f"Index used: {index_used}")
    
    return end_time - start_time

# 1. Define our test queries
def query_courses_by_category():
    """Find all programming courses"""
    return db.courses.find({"category": "Programming"})

def query_active_students():
    """Find all active students"""
    return db.users.find({"role": "student", "isActive": True})

def query_upcoming_assignments():
    """Find assignments due after today"""
    return db.assignments.find({"dueDate": {"$gt": datetime.now()}}).sort("dueDate", 1)

# 2. Test queries before optimization
print("="*20 + " BEFORE OPTIMIZATION " + "="*20)
original_times = []
original_times.append(test_query(query_courses_by_category, "Courses by category"))
original_times.append(test_query(query_active_students, "Active students"))
original_times.append(test_query(query_upcoming_assignments, "Upcoming assignments"))

# 3. Create optimized indexes
print("\n" + "="*20 + " CREATING INDEXES " + "="*20)
category_index = create_index_safely(
    db.courses,
    [("category", 1)],
    "category_optimized"
)

student_status_index = create_index_safely(
    db.users,
    [("role", 1), ("isActive", 1)],
    "active_students_optimized"
)

due_date_index = create_index_safely(
    db.assignments,
    [("dueDate", 1)],
    "due_date_optimized"
)

# 4. Define optimized queries using hints
def optimized_category_query():
    return db.courses.find({"category": "Programming"}).hint(category_index)

def optimized_active_students():
    return db.users.find({"role": "student", "isActive": True}).hint(student_status_index)

def optimized_assignments():
    return db.assignments.find({"dueDate": {"$gt": datetime.now()}}).sort("dueDate", 1).hint(due_date_index)

# 5. Test optimized queries
print("\n" + "="*20 + " AFTER OPTIMIZATION " + "="*20)
optimized_times = []
optimized_times.append(test_query(optimized_category_query, "Courses by category (optimized)"))
optimized_times.append(test_query(optimized_active_students, "Active students (optimized)"))
optimized_times.append(test_query(optimized_assignments, "Upcoming assignments (optimized)"))

# 6. Calculate and display improvements
print("\n" + "="*20 + " PERFORMANCE IMPROVEMENT " + "="*20)
improvements = [
    original_times[0]/optimized_times[0],
    original_times[1]/optimized_times[1],
    original_times[2]/optimized_times[2]
]

print("\nSpeed Improvement Factors:")
print(f"1. Category query: {improvements[0]:.1f}x faster")
print(f"2. Active students: {improvements[1]:.1f}x faster")
print(f"3. Assignments: {improvements[2]:.1f}x faster")

# 7. Show index information
print("\n" + "="*20 + " INDEX INFORMATION " + "="*20)
print("\nCourses indexes:")
pprint(db.courses.index_information())

print("\nUsers indexes:")
pprint(db.users.index_information())

print("\nAssignments indexes:")
pprint(db.assignments.index_information())

==================== BEFORE OPTIMIZATION ====================

COURSES BY CATEGORY RESULTS
----------------------------------------
Execution time: 1.97 ms
Documents examined: 9
Results returned: 1
Index used: None

ACTIVE STUDENTS RESULTS
----------------------------------------
Execution time: 2.02 ms
Documents examined: 21
Results returned: 13
Index used: None

UPCOMING ASSIGNMENTS RESULTS
----------------------------------------
Execution time: 22.09 ms
Documents examined: 0
Results returned: 0
Index used: None

==================== CREATING INDEXES ====================
Created new index: category_optimized
Created new index: active_students_optimized
Using existing index due_date_idx with same specification

==================== AFTER OPTIMIZATION ====================

COURSES BY CATEGORY (OPTIMIZED) RESULTS
----------------------------------------
Execution time: 16.01 ms
Documents examined: 1
Results returned: 1
Index used: None

ACTIVE STUDENTS (OPTIMIZED) RESULTS
-------------

### Task 6.2: Error Handling

Write queries that handle common errors:

1. Duplicate key errors

2. Invalid data type insertions

3. Missing required fields

In [29]:

def handle_errors():
    print("=== ERROR HANDLING DEMONSTRATION ===")
    
    # 1. Handle duplicate key errors
    print("\n1. Duplicate Key Error Handling:")
    try:
        # Try inserting a user with duplicate email
        db.users.insert_one({
            "userId": "user999",
            "email": "adebola.adesanya@gmail.com",  # Existing email
            "firstName": "Test",
            "lastName": "User",
            "role": "student",
            "dateJoined": datetime.now(),
            "isActive": True
        })
    except DuplicateKeyError as e:
        print(f"Caught DuplicateKeyError: {e.details['errmsg']}")
        print("Action: Rejected duplicate email address")
    
    # 2. Handle invalid data type insertions
    print("\n2. Invalid Data Type Handling:")
    try:
        # Try inserting invalid data (string instead of number for price)
        db.courses.insert_one({
            "courseId": "course999",
            "title": "Invalid Course",
            "price": "free",  # Should be number
            "instructorId": "user001",
            "createdAt": datetime.now()
        })
    except OperationFailure as e:
        print(f"Caught OperationFailure: {e.details['errmsg']}")
        print("Action: Rejected invalid price data type")
    
    # 3. Handle missing required fields
    print("\n3. Missing Required Fields Handling:")
    try:
        # Try inserting user without required email field
        db.users.insert_one({
            "userId": "user999",
            "firstName": "Test",
            "lastName": "User",
            "role": "student",
            "dateJoined": datetime.now(),
            "isActive": True
        })
    except OperationFailure as e:
        print(f"Caught OperationFailure: {e.details['errmsg']}")
        print("Action: Rejected document missing required email field")
    
    # 4. Bonus: Document validation error (schema mismatch)
    print("\n4. Schema Validation Handling:")
    try:
        # Try inserting invalid role
        db.users.insert_one({
            "userId": "user999",
            "email": "test.user@example.com",
            "firstName": "Test",
            "lastName": "User",
            "role": "admin",  # Not in enum (student/instructor)
            "dateJoined": datetime.now(),
            "isActive": True
        })
    except OperationFailure as e:
        print(f"Caught OperationFailure: {e.details['errmsg']}")
        print("Action: Rejected invalid role value")

# Run the error handling demonstration
handle_errors()

# Verification of error handling
print("\n=== VERIFICATION ===")
print("1. Duplicate email test:")
print(f"- User with duplicate email exists: {db.users.count_documents({'email': 'adebola.adesanya@gmail.com'}) == 1}")

print("\n2. Invalid course test:")
print(f"- Invalid course was not inserted: {db.courses.count_documents({'courseId': 'course999'}) == 0}")

print("\n3. Missing field test:")
print(f"- User without email was not inserted: {db.users.count_documents({'userId': 'user999'}) == 0}")

print("\n4. Invalid role test:")
print(f"- User with invalid role was not inserted: {db.users.count_documents({'email': 'test.user@example.com'}) == 0}")

=== ERROR HANDLING DEMONSTRATION ===

1. Duplicate Key Error Handling:
Caught DuplicateKeyError: E11000 duplicate key error collection: eduhub_db.users index: email_lookup_idx dup key: { email: "adebola.adesanya@gmail.com" }
Action: Rejected duplicate email address

2. Invalid Data Type Handling:
Caught OperationFailure: Document failed validation
Action: Rejected invalid price data type

3. Missing Required Fields Handling:
Caught OperationFailure: Document failed validation
Action: Rejected document missing required email field

4. Schema Validation Handling:
Caught OperationFailure: Document failed validation
Action: Rejected invalid role value

=== VERIFICATION ===
1. Duplicate email test:
- User with duplicate email exists: True

2. Invalid course test:
- Invalid course was not inserted: True

3. Missing field test:
- User without email was not inserted: True

4. Invalid role test:
- User with invalid role was not inserted: True


In [12]:
client.drop_database('eduhub_db')